LEEME antes de empezar: 

1. Apuntamos hacia el csv que queremos leer (siempre tiene que estar en la misma carpeta, en caso contrario, poner la ruta de donde se encuentra en vuestro PC)
2. De ahí ya está preparado para que nos saque el nombre de peli, el año y el ID de imdb 
3. Hace una concatenación de año y nombre para luego buscarlo en tomatoes. 
4. Empezamos con IMDB: hay Ids que no existen y primero los ignoraba, pero despues los he añadido a reusltados todo con NA para que la LEN de este resultado sea = a la LEN de tomates y podamos fusionar y hacer DF (luego lo depuramos en sql con not null)
5. Tomatitos = saca el valor de tomatometer y lo guarda en una lista 
6. Cuando tengo ambas listas las concateno, /cada tomatometer irá en lapeli que toca ya que hemos añadido tambien los NULL para asegurar el orden 
7. Hacemos el Data Frame con la columna de tomatometer añadida al final 
8. Guardamos CSV con el nombre que toque en cada caso 

¡ÁNIMO!

LO UNICO QUE HAY QUE CAMBIAR EN CADA CASO: 

- df_romance = el csv de la variable df_romance
- df_total.to_csv("romance_fase2_1990_2023.csv") el nombre del csv que guardo 


In [1]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.
import re
from pandas import DataFrame
# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import ast
import json
# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.

from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.common.exceptions import NoSuchElementException

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.


OJO aqui tenemos que cambiar el CSV al cual apuntemos, ahora está el "romance"

In [2]:
#EJEMPLO:cojo mi csv de pelis de romance para testear la iteración por ID en imdb 
#CAMBIAME!! 
df_romance = pd.read_csv("../../../romance_test_fase2.csv", index_col=0)
df_romance.head()

,type,name,year,month,id
genre,,,,,
Romance,Short,Truth,2023,1,tt25593524
Romance,Movie,Heaven Sent,2023,2,tt25598548
Romance,Movie,A Winter Tale at Shimla,2023,5,tt25729566
Romance,Movie,Sofi,2023,1,tt25730144
Romance,Movie,Voyages en Italie,2023,3,tt25750424


In [3]:
lista_ids = df_romance["id"]
lista_ids.head()

genre
Romance    tt25593524
Romance    tt25598548
Romance    tt25729566
Romance    tt25730144
Romance    tt25750424
Name: id, dtype: object

In [4]:
lista_names_pelis = df_romance["name"].values
lista_años_pelis = df_romance["year"].values

In [5]:
lista_año_peli_zip = list(zip(lista_names_pelis, lista_años_pelis))
lista_año_peli_zip

[('Truth', 2023),
 ('Heaven Sent', 2023),
 ('A Winter Tale at Shimla', 2023),
 ('Sofi', 2023),
 ('Voyages en Italie', 2023),
 ('Mareyade Kshamisu', 2023),
 ('(In)convenience', 2023),
 ('Duti Phagun', 2023),
 ('Smarami', 2023),
 ('Love on its Side', 2023),
 ('A Sunday Affair', 2023),
 ('Noches de Abril', 2023),
 ('Romina, VTM', 2023),
 ('What About You', 2023)]

# PARTE 1 -- IMDB

In [6]:
# ABRIMOS NAVEGADOR ------------------------------------------------------

driver = webdriver.Chrome()

#Busca la web que yo quiera, en este caso IMDB
driver.get("https://www.imdb.com/")
#print("0 - Abriendo la web")

# Maximizar la ventana del navegador para mostrar la página completa
driver.maximize_window()


#------------------------- COMENZAMOS LOOP ---------------------------------

lista_imdb = []
for id in tqdm(lista_ids):
    lista_consumible = []
    
    try:

        driver.find_element("css selector", "#suggestion-search").send_keys(id, Keys.ENTER)
        sleep(4)

        lista_consumible.append(id)
        #print(" 3- Añadiendo ID")

#PUNTUACION ----------------------------------------------------------------
        try: 
            puntuacion = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-3a4309f8-0.fjtZsE.sc-dffc6c81-1.fJrHDo > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.gtEgaf > div.sc-bde20123-2.gYgHoj > span.sc-bde20123-1.iZlgcd").text
            #print("4 - Sacando puntuación imdb")
            lista_consumible.append(puntuacion)
            sleep(4)
        except: 
            lista_consumible.append("NA")
            

#DIRECCION ----------------------------------------------------------------
        try: 

            director = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(1) > div > ul > li > a").text
            lista_consumible.append(director)
            #print("4 - Sacando Director")
            sleep(4)
        except: 
            lista_consumible.append("NA")

#GUIONISTAS --------------------------------------------------------------
        
        try: 
            guionistas = []

            for guionista in range(1,10):
                try: 
                    nombres_guionistas = driver.find_element("css selector", f"#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div > ul > li:nth-child({guionista}) > a").text
                    guionistas.append(nombres_guionistas)
                except:
                    break

            lista_consumible.append(guionistas)
        except: 
            lista_consumible.append("NA")
            #print("5 - Sacando Guionistas")

#ARGUMENTO PELICULA -----------------------------------------------------
        try:

            
            argumento = driver.find_element("xpath", "/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]").text
            
        except NoSuchElementException:
            # Handle the case where the element is not found
            argumento = "Element not found"

        lista_consumible.append(argumento)

        sleep(4)

        #print("9 - Sacando Argumento")

#DURACION ---------------------------------------------------------------
        try: 
            duracion_horas = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > ul > li:nth-child(2)").text 
            #print("6 - Sacando duración")
            #PARA CONVERTIR LA DURACION TOTAL A MINUTOS! 
            partes = duracion_horas.split() 

            horas = 0 
            minutos = 0 

            for parte in partes:
                if "h" in parte: 
                    horas = horas + int(parte[:-1]) 
                elif "m" in parte: 
                    minutos = minutos + int(parte[:-1])# :-1 al eliminar el H y el M entonces nos da el digit. Si pongo [0]me daría solo el 2 de 22, o el 2 de 25.

            duracion = horas * 60 + minutos 

            lista_consumible.append(duracion)
            sleep(6)
            #print("7 - convirtiendo a minutos")
        except: 
            lista_consumible.append("NA")

#NOMBRE PELICULA ----------------------------------------------------------------
        try: 

            nombre_imdb = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > h1 > span").text
            sleep(4)
            lista_consumible.append(nombre_imdb)
            
            #print("8 - Sacando nombre Peli")
            sleep(4)
        except:
            lista_consumible.append("NA")

#----------------------------------------------------------------
# He añadido este parche para que me añada los casos de Ids de pelis que ya no existen en imdb pero que los busco igualmente en rotten tomatoes. 
# Luego se puede limpiar en BBDD pero así la len de ambas listas es = para hacer el dataframe y guardar el CSV 

    except NoSuchElementException:
        # Cuando el ID que estamos buscando no existe en imdb
        print(f"ID {id} does not exist on IMDb. Skipping...")
        puntuacion = "NA"
        lista_consumible.append(puntuacion)
        nombre_imdb = "NA"
        lista_consumible.append(nombre_imdb)
        duracion = "NA"
        lista_consumible.append(duracion)
        director = "NA"
        lista_consumible.append(director)
        argumento = "NA"
        lista_consumible.append(argumento)
        guionistas = "NA"
        lista_consumible.append(guionistas)

        driver.back()
        sleep(8)



# FUERA DEL LOOP: 

    if lista_consumible: #evita hacer append de tuplas vacías
                            
        lista_imdb.append(lista_consumible)
        #FALTA CONVERTIRLO A TUPLA


100%|██████████| 14/14 [07:28<00:00, 32.02s/it]


# PARTE 2 - TOMATITOS 

In [7]:

# Website 2 - Rotten Tomatoes

# Inicializa el navegador de Chrome--------------------------
driver = webdriver.Chrome()

# Abre Google en el navegador--------------------------
driver.get("https://www.google.com/")
#print("1 - Abriendo Google")

# Maximiza la ventana del navegador--------------------------
driver.maximize_window()

# Acepta las cookies de Google--------------------------

try:
    driver.find_element(By.CSS_SELECTOR, "#L2AGLb > div").click()
    #print("2 - Cookies de Google")
except NoSuchElementException:
    print("El botón de cookies no se encontró, continuando con el código.")


lista_tomates = []

# Bucle para buscar las películas --------------------------
for año, título in tqdm(lista_año_peli_zip):

# Realiza una búsqueda en Google --------------------------
    driver.get("https://www.google.com/")
    #print("3 - Entrando a Google"

    busqueda_pelicula = f"{año} {título} Rotten Tomatoes"
    # driver.find_element("css selector", "#APjFqb").clear()
    driver.find_element(By.CSS_SELECTOR, "#APjFqb").send_keys(
        busqueda_pelicula, Keys.ENTER
    )

    try:
        element = driver.find_element(
            By.XPATH,
            "/html/body/div[5]/div/div[12]/div/div[1]/div[1]/div/div/div[3]/div/div/div[2]/div/g-raised-button[1]/div/span",
        )

    except NoSuchElementException:
        print("Sign up de google passed.")
        # SI ENTRO AQUI, IRÉ DIRECTAMENTE EL SIGUIENTE TRY (ES COMO UN PASS)
    else:
        element.click()

# Haz clic en el primer resultado de búsqueda ----------------------
    try:
        element = driver.find_element(By.XPATH, "//*[@id='rso']/div[1]/descendant::h3")

    except NoSuchElementException:
        print(
            "No se encontró el elemento. Puede ser necesario actualizar el selector CSS."
        )

        continue

    element.click()

    #print("5 - Haciendo clic en la búsqueda")

# Acepta las cookies en la página de Rotten Tomatoes --------------------------

    try:
        element = driver.find_element(By.CSS_SELECTOR, "#onetrust-accept-btn-handler")

    except NoSuchElementException:
        print("Cookies de rotten")
    # SI ENTRO AQUI, IRÉ DIRECTAMENTE EL SIGUIENTE TRY (ES COMO UN PASS)
    else:
        element.click()

    #print("6 - Aceptando cookies en Rotten Tomatoes")

    try:
# Accede a la URL de Rotten Tomatoes para obtener información --------------------------

        tomatometer_element = driver.find_element("xpath", "//*[@id='scoreboard']")
        tomatometer = tomatometer_element.get_attribute("tomatometerscore")

        if tomatometer:
            lista_tomates.append([f"{tomatometer}%"])
            #print(f"Tomatometer Value for {año} - {título}: {tomatometer}")
        else:
            lista_tomates.append(["--"])
 
            #print("Tomatometer not found")
    except NoSuchElementException:
        lista_tomates.append(["--"])
    
        #print("No se encontró el tomato")

driver.quit()



  0%|          | 0/14 [00:00<?, ?it/s]

Sign up de google passed.


  7%|▋         | 1/14 [00:11<02:24, 11.10s/it]

Cookies de rotten


 14%|█▍        | 2/14 [00:19<01:52,  9.40s/it]

...

Cookies de rotten
Sign up de google passed.


 93%|█████████▎| 13/14 [02:43<00:10, 10.76s/it]

Cookies de rotten
Sign up de google passed.


100%|██████████| 14/14 [02:56<00:00, 12.61s/it]

Cookies de rotten


In [8]:
df_imdb = pd.DataFrame(lista_imdb, columns=["ID", "Puntuación", "Dirección", "Guionistas", "Argumento", "Duración", "Nombre imdb"])
df_imdb

,ID,Puntuación,Dirección,Guionistas,Argumento,Duración,Nombre imdb
0,tt25593524,NA,Nic Shake,"[Sylvie Zaidman, Chloé Von Arx]","Ruth, a young woman living in a not-so-distant...",15,Truth
1,tt25598548,8.1,Joanne Hock,"[Karen Abercrombie, Tara Lynn Marcelle]","Elise, a grandmother hurt by her past, and Pat...",NA,Heaven Sent
2,tt25729566,6.1,Yogesh Verma,[Yogesh Verma],A Mature Sensible Love Story Where A Man Meets...,140,A Winter Tale at Shimla
3,tt25730144,5.7,Joby Vayalaunkal,"[Olla Prakash, Joby Vayalaunkal]","A love story between a young man, Jo and an ol...",130,Sofi
4,tt25750424,6.0,Sophie Letourneur,"[Laetitia Goffi, Sophie Letourneur]",A couple takes a trip to Sicily to reignite th...,91,Voyages en Italie
5,tt25814516,9.3,K. Raghav,[],A lower-middle-class mortar worker Manja leads...,NA,Mareyade Kshamisu
6,tt25812034,NA,Alexandra Greenspan,[Alexandra Greenspan],"Our short film follows Kirby (he/him), a trans...",12,(In)convenience
7,tt25813778,8.0,NA,[],Avro and Ruhi both same age very good friends ...,15,Duti Phagun
8,tt25815968,9.4,Shivaramakrishna,[Shivaramakrishna],Shiva experiences love in the Utopian world wi...,138,Smarami
9,tt25829298,NA,Taz Ayoub,[Taz Ayoub],Two friends discover love through significant ...,NA,Love on its Side


In [9]:
len(lista_imdb)

14

In [10]:
len(lista_tomates)

14

In [11]:
df_tomates = pd.DataFrame(lista_tomates, columns=["tomatometer"])
df_tomates

,tomatometer
0,--
1,--
2,--
3,--
4,96%
5,--
6,--
7,--
8,--
9,72%



FUSIONAMOS AMBAS LISTAS DE RESULTADOS 

In [12]:
lista_imdb 
lista_tomates 
listas_imdb_tomate = []

for imdb, tomates in zip(lista_imdb, lista_tomates):
    lista_combinada = imdb + tomates
    listas_imdb_tomate.append(tuple(lista_combinada))

print(listas_imdb_tomate)

[('tt25593524', 'NA', 'Nic Shake', ['Sylvie Zaidman', 'Chloé Von Arx'], 'Ruth, a young woman living in a not-so-distant future, seems to live a perfect life. However, when the technology fails, she will have to combat her inner fears to survive, in a world where nothing is what it seems.', 15, 'Truth', '--'), ('tt25598548', '8.1', 'Joanne Hock', ['Karen Abercrombie', 'Tara Lynn Marcelle'], 'Elise, a grandmother hurt by her past, and Patrick, a widowed pastor, have closed the door on love. But with faith and a little help from technology, will their live-streamed romance find a way?', 'NA', 'Heaven Sent', '--'), ('tt25729566', '6.1', 'Yogesh Verma', ['Yogesh Verma'], 'A Mature Sensible Love Story Where A Man Meets His Long Lost Love After Two Decades .The rekindled affair could be ruinous, for the jealous husband who is a DIG Police Officer, a brute with a unenviable reputation.', 140, 'A Winter Tale at Shimla', '--'), ('tt25730144', '5.7', 'Joby Vayalaunkal', ['Olla Prakash', 'Joby Vay

In [13]:

listas_imdb_tomate

[('tt25593524',
  'NA',
  'Nic Shake',
  ['Sylvie Zaidman', 'Chloé Von Arx'],
  'Ruth, a young woman living in a not-so-distant future, seems to live a perfect life. However, when the technology fails, she will have to combat her inner fears to survive, in a world where nothing is what it seems.',
  15,
  'Truth',
  '--'),
 ('tt25598548',
  '8.1',
  'Joanne Hock',
  ['Karen Abercrombie', 'Tara Lynn Marcelle'],
  'Elise, a grandmother hurt by her past, and Patrick, a widowed pastor, have closed the door on love. But with faith and a little help from technology, will their live-streamed romance find a way?',
  'NA',
  'Heaven Sent',
  '--'),
 ('tt25729566',
  '6.1',
  'Yogesh Verma',
  ['Yogesh Verma'],
  'A Mature Sensible Love Story Where A Man Meets His Long Lost Love After Two Decades .The rekindled affair could be ruinous, for the jealous husband who is a DIG Police Officer, a brute with a unenviable reputation.',
  140,
  'A Winter Tale at Shimla',
  '--'),
 ('tt25730144',
  '5.7',


YA TENEMOS TOMATOMETER AÑADIDO 

In [14]:
df_total = pd.DataFrame(listas_imdb_tomate, columns=["ID", "Puntuación", "Dirección", "Guionistas", "Argumento", "Duración", "Nombre imdb", "tomatometer"])
df_total

,ID,Puntuación,Dirección,Guionistas,Argumento,Duración,Nombre imdb,tomatometer
0,tt25593524,NA,Nic Shake,"[Sylvie Zaidman, Chloé Von Arx]","Ruth, a young woman living in a not-so-distant...",15,Truth,--
1,tt25598548,8.1,Joanne Hock,"[Karen Abercrombie, Tara Lynn Marcelle]","Elise, a grandmother hurt by her past, and Pat...",NA,Heaven Sent,--
2,tt25729566,6.1,Yogesh Verma,[Yogesh Verma],A Mature Sensible Love Story Where A Man Meets...,140,A Winter Tale at Shimla,--
3,tt25730144,5.7,Joby Vayalaunkal,"[Olla Prakash, Joby Vayalaunkal]","A love story between a young man, Jo and an ol...",130,Sofi,--
4,tt25750424,6.0,Sophie Letourneur,"[Laetitia Goffi, Sophie Letourneur]",A couple takes a trip to Sicily to reignite th...,91,Voyages en Italie,96%
5,tt25814516,9.3,K. Raghav,[],A lower-middle-class mortar worker Manja leads...,NA,Mareyade Kshamisu,--
6,tt25812034,NA,Alexandra Greenspan,[Alexandra Greenspan],"Our short film follows Kirby (he/him), a trans...",12,(In)convenience,--
7,tt25813778,8.0,NA,[],Avro and Ruhi both same age very good friends ...,15,Duti Phagun,--
8,tt25815968,9.4,Shivaramakrishna,[Shivaramakrishna],Shiva experiences love in the Utopian world wi...,138,Smarami,--
9,tt25829298,NA,Taz Ayoub,[Taz Ayoub],Two friends discover love through significant ...,NA,Love on its Side,72%


Guardar todo lo que hagamos con nombre de genero + fase + año inicio + año fin

#vamos cambiando el nombre del csv por cada tramo de pelis que hagamos

In [15]:
df_total.to_csv("romance_fase2_ejemplo.csv")